In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage
import os




path = r"C:\Users\cory\Documents\ddsm-samples"
ren = os.listdir(path)
length = len(ren)

keypoints = np.zeros((length, 9))

for m in range(0, length):
    addition = os.path.join(path,ren[m])
    image = cv2.imread(addition,0)
    Beginning_image = image
    print (ren[m])
   

    def Image_check_x(image):
        p = image.shape[0]
        if ((p+3)/4) != int((p+3)/4):
            if ((p+2)/4) != int((p+2)/4):
                if ((p+1)/4) != int((p+1)/4):
                    return p
                else:
                    return p+1
            else:
                return p+2
        else:
            return p+3



    def Image_check_y(image):
        p = image.shape[1]
        if ((p+3)/4) != int((p+3)/4):
            if ((p+2)/4) != int((p+2)/4):
                if ((p+1)/4) != int((p+1)/4):
                    return p
                else:
                    return p+1
            else:
                return p+2
        else:
            return p+3



    
    
    q = int(Image_check_x(Beginning_image))
    u = int(Image_check_y(Beginning_image))
    beginning_image = cv2.resize(Beginning_image,(u,q))
    s = 3
    k = 2 ** (1.0 / s) #kernals
    sig = 1.6
    kvector = np.array([sig, k*sig, sig*(k**2), sig*(k**3), sig*(k**4), sig*(k**5),sig*(k**6), sig*(k**7), sig*(k**8)])
    c = -1
    threshold = 1

    def gauss_blur(sigma):

        """Function to mimic the 'fspecial' gaussian MATLAB function
        """
        size = 2*np.ceil(3*sigma)+1
        x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
        g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
        return g/g.sum()


    #create image size variant
    doubled = cv2.resize((beginning_image), ((2*u),(2*q)))
    normal = cv2.resize((doubled),(u,q))
    half = cv2.resize((normal), (int(u/2),int(q/2)))
    quarter = cv2.resize((half),(int(u/4),int(q/4)))
    
    

    #create zero layer for pyramid
    pyrlvl1 = np.zeros((doubled.shape[0], doubled.shape[1],6))
    pyrlvl2 = np.zeros((normal.shape[0], normal.shape[1],6))
    pyrlvl3 = np.zeros((half.shape[0], half.shape[1],6))
    pyrlvl4 = np.zeros((quarter.shape[0], quarter.shape[1],6))

    # create our Gaussian pyramid
    for i in range(0,6):
        pyrlvl1[:,:,i] = ndimage.filters.gaussian_filter(doubled, kvector[i]) 
        pyrlvl2[:,:,i] = ndimage.filters.gaussian_filter(normal, kvector[i+1])
        pyrlvl3[:,:,i] = ndimage.filters.gaussian_filter(half, kvector[i+2])
        pyrlvl4[:,:,i] = ndimage.filters.gaussian_filter(quarter,kvector[i+3])

    #create zero layer for DoG images 
    DoGlvl1 = np.zeros((doubled.shape[0], doubled.shape[1],5))
    DoGlvl2 = np.zeros((normal.shape[0], normal.shape[1],5))
    DoGlvl3 = np.zeros((half.shape[0], half.shape[1],5))
    DoGlvl4 = np.zeros((quarter.shape[0], quarter.shape[1],5))

    # create DoG layer
    for i in range (0,5):
        DoGlvl1[:,:,i] = pyrlvl1[:,:,i+1] - pyrlvl1[:,:,i]
        DoGlvl2[:,:,i] = pyrlvl2[:,:,i+1] - pyrlvl2[:,:,i]
        DoGlvl3[:,:,i] = pyrlvl3[:,:,i+1] - pyrlvl3[:,:,i]
        DoGlvl4[:,:,i] = pyrlvl4[:,:,i+1] - pyrlvl4[:,:,i]
    
    # create zero layer to store extrema location
    Exlvl1 = np.zeros((doubled.shape[0], doubled.shape[1],3))
    Exlvl2 = np.zeros((normal.shape[0], normal.shape[1],3))
    Exlvl3 = np.zeros((half.shape[0], half.shape[1],3))
    Exlvl4 = np.zeros((quarter.shape[0], quarter.shape[1],3))

    area1 = np.zeros((doubled.shape[0], doubled.shape[1]))
    area2 = np.zeros((normal.shape[0], normal.shape[1]))
    area3 = np.zeros((half.shape[0], half.shape[1]))
    area4 = np.zeros((quarter.shape[0], quarter.shape[1]))

    for j in range(1, doubled.shape[0] -1):
        for k in range(1, doubled.shape[1]-1):
            if pyrlvl1[j,k,1]> 0:
                area1[j,k] = 1

    for j in range(1, normal.shape[0] -1):
        for k in range(1, normal.shape[1]-1):
            if pyrlvl2[j,k,1]> 0:
                area2[j,k] = 1
            
    for j in range(1, half.shape[0] -1):
        for k in range(1, half.shape[1]-1):
            if pyrlvl3[j,k,1]> 0:
                area3[j,k] = 1
            
    for j in range(1, quarter.shape[0] -1):
        for k in range(1, quarter.shape[1]-1):
            if pyrlvl4[j,k,1]> 0:
                area4[j,k] = 1
            
    for i in range(1,4):
        for j in range(16, doubled.shape[0] -16):
            for k in range(16, doubled.shape[1]-16):
                if np.absolute(DoGlvl1[j, k, i]) < threshold:
                    continue
                maxima = DoGlvl1[j,k,i] > 0
                minima = DoGlvl1[j,k,i] < 0
                for di in range(-1,2):
                    for dj in range(-1,2):
                        for dk in range(-1,2):
                            if di == 0 and dj ==0 and dk == 0:
                                continue
                            maxima = maxima and (DoGlvl1[j, k, i] > DoGlvl1[j + dj, k + dk, i + di])
                            minima = minima and (DoGlvl1[j, k, i] < DoGlvl1[j + dj, k + dk, i + di])
                        
                            if not maxima and not minima:
                                break

                        if not maxima and not minima:
                            break

                    if not maxima and not minima:
                        break
                if maxima or minima:
                    dx = (DoGlvl1[j, k+1, i] - DoGlvl1[j, k-1, i]) * 0.5 / 255
                    dy = (DoGlvl1[j+1, k, i] - DoGlvl1[j-1, k, i]) * 0.5 / 255
                    dz = (DoGlvl1[j, k, i+1] - DoGlvl1[j, k, i-1]) * 0.5 / 255
                    dxx = (DoGlvl1[j, k+1, i] + DoGlvl1[j, k-1, i] - 2 * DoGlvl1[j, k, i]) * 1.0 / 255        
                    dyy = (DoGlvl1[j+1, k, i] + DoGlvl1[j-1, k, i] - 2 * DoGlvl1[j, k, i]) * 1.0 / 255          
                    dzz = (DoGlvl1[j, k, i+1] + DoGlvl1[j, k, i-1] - 2 * DoGlvl1[j, k, i]) * 1.0 / 255
                    dxy = (DoGlvl1[j+1, k+1, i] - DoGlvl1[j+1, k-1, i] - DoGlvl1[j-1, k+1, i] + DoGlvl1[j-1, k-1, i]) * 0.25 / 255 
                    dxz = (DoGlvl1[j, k+1, i+1] - DoGlvl1[j, k-1, i+1] - DoGlvl1[j, k+1, i-1] + DoGlvl1[j, k-1, i-1]) * 0.25 / 255 
                    dyz = (DoGlvl1[j+1, k, i+1] - DoGlvl1[j-1, k, i+1] - DoGlvl1[j+1, k, i-1] + DoGlvl1[j-1, k, i-1]) * 0.25 / 255  
                
                
                    J = np.matrix([[dx], [dy], [dz]])
                    DH = np.matrix([[dxx, dxy, dxz], [dxy, dyy, dyz], [dxz, dyz, dzz]])
                    detA = (dxx*((dyy * dzz)-(dyz**2))) + (dxy*((dxy*dzz)-(dxz*dyz))) + (dxz*((dxy*dxz)-((dyy*dxz))))
                    invDH = np.linalg.pinv(DH)
                
                    X_hat = np.dot(invDH,J)
                    D_X_hat = DoGlvl1[j,k,i] + (0.5 * np.dot(J,np.transpose(X_hat))) #contast variable
                    r = 10.0
                    if [(((dxx + dyy) ** 2) * r) < (dxx * dyy - (dxy ** 2)) * (((r + 1) ** 2)) and np.absolute(D_X_hat) < 0.03]:
                        Exlvl1[j, k, i - 1] = 1

    for i in range(1,4):
        for j in range(8, normal.shape[0] -8):
            for k in range(8, normal.shape[1]-8):
                if np.absolute(DoGlvl2[j, k, i]) < threshold:
                    continue
                maxima = DoGlvl2[j,k,i] > 0
                minima = DoGlvl2[j,k,i] < 0
                for di in range(-1,2):
                    for dj in range(-1,2):
                        for dk in range(-1,2):
                            if di == 0 and dj ==0 and dk == 0:
                                continue
                            maxima = maxima and (DoGlvl2[j, k, i] > DoGlvl2[j + dj, k + dk, i + di])
                            minima = minima and (DoGlvl2[j, k, i] < DoGlvl2[j + dj, k + dk, i + di])
                        
                            if not maxima and not minima:
                                break

                        if not maxima and not minima:
                            break

                    if not maxima and not minima:
                        break
                if maxima or minima:
                    dx = (DoGlvl2[j, k+1, i] - DoGlvl2[j, k-1, i]) * 0.5 / 255
                    dy = (DoGlvl2[j+1, k, i] - DoGlvl2[j-1, k, i]) * 0.5 / 255
                    dz = (DoGlvl2[j, k, i+1] - DoGlvl2[j, k, i-1]) * 0.5 / 255
                    dxx = (DoGlvl2[j, k+1, i] + DoGlvl2[j, k-1, i] - 2 * DoGlvl2[j, k, i]) * 1.0 / 255        
                    dyy = (DoGlvl2[j+1, k, i] + DoGlvl2[j-1, k, i] - 2 * DoGlvl2[j, k, i]) * 1.0 / 255          
                    dzz = (DoGlvl2[j, k, i+1] + DoGlvl2[j, k, i-1] - 2 * DoGlvl2[j, k, i]) * 1.0 / 255
                    dxy = (DoGlvl2[j+1, k+1, i] - DoGlvl2[j+1, k-1, i] - DoGlvl2[j-1, k+1, i] + DoGlvl2[j-1, k-1, i]) * 0.25 / 255 
                    dxz = (DoGlvl2[j, k+1, i+1] - DoGlvl2[j, k-1, i+1] - DoGlvl2[j, k+1, i-1] + DoGlvl2[j, k-1, i-1]) * 0.25 / 255 
                    dyz = (DoGlvl2[j+1, k, i+1] - DoGlvl2[j-1, k, i+1] - DoGlvl2[j+1, k, i-1] + DoGlvl2[j-1, k, i-1]) * 0.25 / 255  
                
                
                    J = np.matrix([[dx], [dy], [dz]])
                    DH = np.matrix([[dxx, dxy, dxz], [dxy, dyy, dyz], [dxz, dyz, dzz]])
                    detA = (dxx*((dyy * dzz)-(dyz**2))) + (dxy*((dxy*dzz)-(dxz*dyz))) + (dxz*((dxy*dxz)-((dyy*dxz))))
                    invDH = np.linalg.pinv(DH)
                    X_hat = np.dot(invDH,J)
                    D_X_hat = DoGlvl2[j,k,i] + (0.5 * np.dot(J,np.transpose(X_hat))) #contast variable
                    r = 10.0
                    if [(((dxx + dyy) ** 2) * r) < (dxx * dyy - (dxy ** 2)) * (((r + 1) ** 2)) and np.absolute(D_X_hat) < 0.03]:
                        Exlvl2[j, k, i - 1] = 1
                    
    for i in range(1,4):
        for j in range(4, half.shape[0] -4):
            for k in range(4, half.shape[1]-4):
                if np.absolute(DoGlvl3[j, k, i]) < threshold:
                    continue
                maxima = DoGlvl3[j,k,i] > 0
                minima = DoGlvl3[j,k,i] < 0
                for di in range(-1,2):
                    for dj in range(-1,2):
                        for dk in range(-1,2):
                            if di == 0 and dj ==0 and dk == 0:
                                continue
                            maxima = maxima and (DoGlvl3[j, k, i] > DoGlvl3[j + dj, k + dk, i + di])
                            minima = minima and (DoGlvl3[j, k, i] < DoGlvl3[j + dj, k + dk, i + di])
                        
                            if not maxima and not minima:
                                break

                        if not maxima and not minima:
                            break

                    if not maxima and not minima:
                        break
                if maxima or minima:
                    dx = (DoGlvl3[j, k+1, i] - DoGlvl3[j, k-1, i]) * 0.5 / 255
                    dy = (DoGlvl3[j+1, k, i] - DoGlvl3[j-1, k, i]) * 0.5 / 255
                    dz = (DoGlvl3[j, k, i+1] - DoGlvl3[j, k, i-1]) * 0.5 / 255
                    dxx = (DoGlvl3[j, k+1, i] + DoGlvl3[j, k-1, i] - 2 * DoGlvl3[j, k, i]) * 1.0 / 255        
                    dyy = (DoGlvl3[j+1, k, i] + DoGlvl3[j-1, k, i] - 2 * DoGlvl3[j, k, i]) * 1.0 / 255          
                    dzz = (DoGlvl3[j, k, i+1] + DoGlvl3[j, k, i-1] - 2 * DoGlvl3[j, k, i]) * 1.0 / 255
                    dxy = (DoGlvl3[j+1, k+1, i] - DoGlvl3[j+1, k-1, i] - DoGlvl3[j-1, k+1, i] + DoGlvl3[j-1, k-1, i]) * 0.25 / 255 
                    dxz = (DoGlvl3[j, k+1, i+1] - DoGlvl3[j, k-1, i+1] - DoGlvl3[j, k+1, i-1] + DoGlvl3[j, k-1, i-1]) * 0.25 / 255 
                    dyz = (DoGlvl3[j+1, k, i+1] - DoGlvl3[j-1, k, i+1] - DoGlvl3[j+1, k, i-1] + DoGlvl3[j-1, k, i-1]) * 0.25 / 255  
                
                
                    J = np.matrix([[dx], [dy], [dz]])
                    DH = np.matrix([[dxx, dxy, dxz], [dxy, dyy, dyz], [dxz, dyz, dzz]])
                    detA = (dxx*((dyy * dzz)-(dyz**2))) + (dxy*((dxy*dzz)-(dxz*dyz))) + (dxz*((dxy*dxz)-((dyy*dxz))))
                    invDH = np.linalg.pinv(DH)
                
                    X_hat = np.dot(invDH,J)
                    D_X_hat = DoGlvl3[j,k,i] + (0.5 * np.dot(J,np.transpose(X_hat))) #contast variable
                    r = 10.0
                    if [(((dxx + dyy) ** 2) * r) < (dxx * dyy - (dxy ** 2)) * (((r + 1) ** 2)) and np.absolute(D_X_hat) < 0.03]:
                        Exlvl3[j, k, i - 1] = 1
        
    for i in range(1,4):
        for j in range(2, quarter.shape[0] -2):
            for k in range(2, quarter.shape[1]-2):
                if np.absolute(DoGlvl4[j, k, i]) < threshold:
                    continue
                maxima = DoGlvl4[j,k,i] > 0
                minima = DoGlvl4[j,k,i] < 0
                for di in range(-1,2):
                    for dj in range(-1,2):
                        for dk in range(-1,2):
                            if di == 0 and dj ==0 and dk == 0:
                                continue
                            maxima = maxima and (DoGlvl4[j, k, i] > DoGlvl4[j + dj, k + dk, i + di])
                            minima = minima and (DoGlvl4[j, k, i] < DoGlvl4[j + dj, k + dk, i + di])
                        
                            if not maxima and not minima:
                                break

                        if not maxima and not minima:
                            break

                    if not maxima and not minima:
                        break
                if maxima or minima:
                    dx = (DoGlvl4[j, k+1, i] - DoGlvl4[j, k-1, i]) * 0.5 / 255
                    dy = (DoGlvl4[j+1, k, i] - DoGlvl4[j-1, k, i]) * 0.5 / 255
                    dz = (DoGlvl4[j, k, i+1] - DoGlvl4[j, k, i-1]) * 0.5 / 255
                    dxx = (DoGlvl4[j, k+1, i] + DoGlvl4[j, k-1, i] - 2 * DoGlvl4[j, k, i]) * 1.0 / 255        
                    dyy = (DoGlvl4[j+1, k, i] + DoGlvl4[j-1, k, i] - 2 * DoGlvl4[j, k, i]) * 1.0 / 255          
                    dzz = (DoGlvl4[j, k, i+1] + DoGlvl4[j, k, i-1] - 2 * DoGlvl4[j, k, i]) * 1.0 / 255
                    dxy = (DoGlvl4[j+1, k+1, i] - DoGlvl4[j+1, k-1, i] - DoGlvl4[j-1, k+1, i] + DoGlvl4[j-1, k-1, i]) * 0.25 / 255 
                    dxz = (DoGlvl4[j, k+1, i+1] - DoGlvl4[j, k-1, i+1] - DoGlvl4[j, k+1, i-1] + DoGlvl4[j, k-1, i-1]) * 0.25 / 255 
                    dyz = (DoGlvl4[j+1, k, i+1] - DoGlvl4[j-1, k, i+1] - DoGlvl4[j+1, k, i-1] + DoGlvl4[j-1, k, i-1]) * 0.25 / 255  
                
                
                    J = np.matrix([[dx], [dy], [dz]])
                    DH = np.matrix([[dxx, dxy, dxz], [dxy, dyy, dyz], [dxz, dyz, dzz]])
                    detA = (dxx*((dyy * dzz)-(dyz**2))) + (dxy*((dxy*dzz)-(dxz*dyz))) + (dxz*((dxy*dxz)-((dyy*dxz))))
                    invDH = np.linalg.pinv(DH)
                
                    X_hat = np.dot(invDH,J)
                    D_X_hat = DoGlvl4[j,k,i] + (0.5 * np.dot(J,np.transpose(X_hat))) #contast variable
                    r = 10.0
                    if [(((dxx + dyy) ** 2) * r) < (dxx * dyy - (dxy ** 2)) * (((r + 1) ** 2)) and np.absolute(D_X_hat) < 0.03]:
                        Exlvl4[j, k, i - 1] = 1
                    
    print("Number of extrema in first octave: %d" % np.sum(Exlvl1))
    print("Number of extrema in second octave: %d" % np.sum(Exlvl2))
    print("Number of extrema in third octave: %d" % np.sum(Exlvl3))
    print("Number of extrema in fourth octave: %d" % np.sum(Exlvl4))

    extr_sum = int(np.sum(Exlvl1)) + int(np.sum(Exlvl2)) + int(np.sum(Exlvl3)) + int(np.sum(Exlvl4))

    keypoints[m, :] = np.array([[np.sum(Exlvl1) , np.sum(Exlvl2) , np.sum(Exlvl3),np.sum(Exlvl4),extr_sum, np.sum(area1), np.sum(area2), np.sum(area3),np.sum(area4)]])
    print("Calculating keypoint orientations...")

np.savetxt(r"C:\Users\cory\Documents\ddsm-feature-keypoints.csv", keypoints, delimiter=',', fmt='%d')
np.savetxt(r"C:\Users\cory\Documents\ddsm-feature-order.txt",ren,delimiter=',', fmt='%s' )    

p_0001_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2117
Number of extrema in second octave: 240
Number of extrema in third octave: 47
Number of extrema in fourth octave: 12
Calculating keypoint orientations...
p_0002_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 4946
Number of extrema in second octave: 873
Number of extrema in third octave: 147
Number of extrema in fourth octave: 22
Calculating keypoint orientations...
p_0003_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 7028
Number of extrema in second octave: 1014
Number of extrema in third octave: 187
Number of extrema in fourth octave: 15
Calculating keypoint orientations...
p_0004_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 1783
Number of extrema in second octave: 261
Number of extrema in third octave: 27
Number of extrema in fourth octave: 12
Calculating keypoint orientations...
p_0006_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 4306
Number of extrema in second octave: 661
Number of e

Number of extrema in first octave: 1234
Number of extrema in second octave: 162
Number of extrema in third octave: 21
Number of extrema in fourth octave: 3
Calculating keypoint orientations...
p_0230_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2769
Number of extrema in second octave: 444
Number of extrema in third octave: 82
Number of extrema in fourth octave: 15
Calculating keypoint orientations...
p_0231_1.RIGHT_MLO_b1.tif
Number of extrema in first octave: 2838
Number of extrema in second octave: 416
Number of extrema in third octave: 62
Number of extrema in fourth octave: 3
Calculating keypoint orientations...
p_0232_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 2008
Number of extrema in second octave: 271
Number of extrema in third octave: 24
Number of extrema in fourth octave: 8
Calculating keypoint orientations...
p_0235_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 3603
Number of extrema in second octave: 514
Number of extrema in third octave: 85
Numbe

Number of extrema in first octave: 3021
Number of extrema in second octave: 410
Number of extrema in third octave: 57
Number of extrema in fourth octave: 8
Calculating keypoint orientations...
p_0281_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 3574
Number of extrema in second octave: 612
Number of extrema in third octave: 104
Number of extrema in fourth octave: 10
Calculating keypoint orientations...
p_0282_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 1856
Number of extrema in second octave: 246
Number of extrema in third octave: 30
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_0283_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2920
Number of extrema in second octave: 429
Number of extrema in third octave: 47
Number of extrema in fourth octave: 11
Calculating keypoint orientations...
p_0284_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 1725
Number of extrema in second octave: 211
Number of extrema in third octave: 27
Num

Number of extrema in first octave: 4081
Number of extrema in second octave: 709
Number of extrema in third octave: 55
Number of extrema in fourth octave: 18
Calculating keypoint orientations...
p_0343_1.RIGHT_MLO_b4.tif
Number of extrema in first octave: 1473
Number of extrema in second octave: 202
Number of extrema in third octave: 22
Number of extrema in fourth octave: 13
Calculating keypoint orientations...
p_0344_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 882
Number of extrema in second octave: 122
Number of extrema in third octave: 16
Number of extrema in fourth octave: 9
Calculating keypoint orientations...
p_0345_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 3338
Number of extrema in second octave: 531
Number of extrema in third octave: 102
Number of extrema in fourth octave: 20
Calculating keypoint orientations...
p_0350_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 1297
Number of extrema in second octave: 156
Number of extrema in third octave: 31
Num

Number of extrema in first octave: 3692
Number of extrema in second octave: 648
Number of extrema in third octave: 114
Number of extrema in fourth octave: 17
Calculating keypoint orientations...
p_0509_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 1218
Number of extrema in second octave: 170
Number of extrema in third octave: 26
Number of extrema in fourth octave: 3
Calculating keypoint orientations...
p_0511_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2054
Number of extrema in second octave: 318
Number of extrema in third octave: 52
Number of extrema in fourth octave: 8
Calculating keypoint orientations...
p_1108_1.RIGHT_MLO_b1.tif
Number of extrema in first octave: 7340
Number of extrema in second octave: 1273
Number of extrema in third octave: 215
Number of extrema in fourth octave: 24
Calculating keypoint orientations...
p_1109_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 7506
Number of extrema in second octave: 1087
Number of extrema in third octave: 149

...

Number of extrema in first octave: 2455
Number of extrema in second octave: 312
Number of extrema in third octave: 31
Number of extrema in fourth octave: 8
Calculating keypoint orientations...
p_4013_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 2760
Number of extrema in second octave: 298
Number of extrema in third octave: 30
Number of extrema in fourth octave: 13
Calculating keypoint orientations...
p_4014_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 4920
Number of extrema in second octave: 765
Number of extrema in third octave: 119
Number of extrema in fourth octave: 14
Calculating keypoint orientations...
p_4015_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2954
Number of extrema in second octave: 446
Number of extrema in third octave: 71
Number of extrema in fourth octave: 20
Calculating keypoint orientations...
p_4016_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 2633
Number of extrema in second octave: 349
Number of extrema in third octave: 43
Nu

Number of extrema in first octave: 3985
Number of extrema in second octave: 569
Number of extrema in third octave: 88
Number of extrema in fourth octave: 16
Calculating keypoint orientations...
p_4060_1.RIGHT_MLO_b1.tif
Number of extrema in first octave: 3770
Number of extrema in second octave: 560
Number of extrema in third octave: 85
Number of extrema in fourth octave: 14
Calculating keypoint orientations...
p_4061_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 3740
Number of extrema in second octave: 456
Number of extrema in third octave: 63
Number of extrema in fourth octave: 17
Calculating keypoint orientations...
p_4062_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2709
Number of extrema in second octave: 412
Number of extrema in third octave: 51
Number of extrema in fourth octave: 11
Calculating keypoint orientations...
p_4063_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 2625
Number of extrema in second octave: 356
Number of extrema in third octave: 53
Nu

Number of extrema in first octave: 3375
Number of extrema in second octave: 505
Number of extrema in third octave: 60
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_4102_1.RIGHT_MLO_b4.tif
Number of extrema in first octave: 3018
Number of extrema in second octave: 376
Number of extrema in third octave: 28
Number of extrema in fourth octave: 7
Calculating keypoint orientations...
p_4124_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 6161
Number of extrema in second octave: 855
Number of extrema in third octave: 133
Number of extrema in fourth octave: 11
Calculating keypoint orientations...
p_4130_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 2038
Number of extrema in second octave: 252
Number of extrema in third octave: 19
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_4140_1.RIGHT_MLO_b1.tif
Number of extrema in first octave: 4482
Number of extrema in second octave: 745
Number of extrema in third octave: 99
Numb

Number of extrema in first octave: 1872
Number of extrema in second octave: 215
Number of extrema in third octave: 30
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_4510_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 6968
Number of extrema in second octave: 1096
Number of extrema in third octave: 135
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_4511_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 7771
Number of extrema in second octave: 1070
Number of extrema in third octave: 144
Number of extrema in fourth octave: 9
Calculating keypoint orientations...
p_4512_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 3453
Number of extrema in second octave: 451
Number of extrema in third octave: 58
Number of extrema in fourth octave: 11
Calculating keypoint orientations...
p_4513_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 2803
Number of extrema in second octave: 376
Number of extrema in third octave: 33
N

Number of extrema in first octave: 2791
Number of extrema in second octave: 402
Number of extrema in third octave: 47
Number of extrema in fourth octave: 12
Calculating keypoint orientations...
p_4553_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 4264
Number of extrema in second octave: 596
Number of extrema in third octave: 59
Number of extrema in fourth octave: 9
Calculating keypoint orientations...
p_4555_1.RIGHT_MLO_b3.tif
Number of extrema in first octave: 4141
Number of extrema in second octave: 545
Number of extrema in third octave: 62
Number of extrema in fourth octave: 4
Calculating keypoint orientations...
p_4557_1.RIGHT_MLO_b4.tif
Number of extrema in first octave: 1664
Number of extrema in second octave: 161
Number of extrema in third octave: 16
Number of extrema in fourth octave: 6
Calculating keypoint orientations...
p_4558_1.RIGHT_MLO_b2.tif
Number of extrema in first octave: 3568
Number of extrema in second octave: 430
Number of extrema in third octave: 79
Numbe